<h1>Capstone Project - The Battle of Neighbourhoods</h1>

<h2> Introduction </h2>
Canada is one of the most popular places for immigration for Hong Kong citizens. The Chinese Canadian population has been growing since 1960s due to the changes in immigration law and political issues in Hong Kong. The Chinese has established many shops and restuarants to cater their population. As the capital of the province of Ontario, many immigrants favour Toronto as their port of entry. In this project, we would be finding the neighbourhood with the best Chinese resturants in Toronto. This would be valuable for new comers joining the community who prefer to have Chinese food.


<h2> Data </h2>
For this project we need the following data :

* **Toronto city data** that contains list Boroughs, Neighborhoods along with their latitude and longitude.
Data source : http://cocl.us/Geospatial_data
<br/>Description : This data set contains the required information. And we will use this data set to explore various neighborhoods of Toronto city.
Chinese resturants in each neighborhood of Toronto.

* **Fousquare API**
<br/>Description : By using this api we will get all the venues in each neighborhood. We can filter these venues to get only Chinese resturants.

* **City of Toronto Neighborhood Shapes for mapping**
<br/>Data source : https://ckan0.cf.opendata.inter.sandbox-toronto.ca/download_resource/1d02b0f0-d735-4469-8f71-ea6d96b319e4?format=geojson&projection=4326
<br/> Description : By using this data we will get the Toronto neighbourhood boundaries that will help us visualize choropleth map.
